# Check batch key

In [2]:
import warnings
import os
import sys
import gc
import warnings
import anndata as ad
import scanpy as sc
import copy
import torch
from pathlib import Path
import networkx as nx
from sklearn.neighbors import kneighbors_graph
import numpy as np
import scanpy as sc
import pandas as pd
import numpy as np
import scipy.stats

sys.path.append("/home/icb/kemal.inecik/work/codes/tardis")
import tardis
tardis.config = tardis.config_server
print(f"CUDA used: {torch.cuda.is_available()}")

CUDA used: True


In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import matplotlib.pyplot as plt
import seaborn as sns

import pickle
_rcparams_path = "/home/icb/kemal.inecik/work/codes/tardis/training/local/figures/rcparams.pickle"
with open(_rcparams_path, 'rb') as file:
    _rcparams = pickle.load(file)
plt.rcParams.update(_rcparams)

In [33]:
choosen_global_cofounders = dict()

## Miller

In [34]:
dataset = "Miller"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
adata.obs.groupby(['integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size()

integration_donor  integration_biological_unit  integration_sample_status  integration_library_platform_coarse
S1_Miller_et_al    Cell                         Fresh                      3GEX                                    326
S2_Miller_et_al    Cell                         Fresh                      3GEX                                    381
S3_Miller_et_al    Cell                         Fresh                      3GEX                                    683
S4_Miller_et_al    Cell                         Fresh                      3GEX                                    306
S5_Miller_et_al    Cell                         Fresh                      3GEX                                   1935
S6_Miller_et_al    Cell                         Fresh                      3GEX                                   1866
S7_Miller_et_al    Cell                         Fresh                      3GEX                                   1139
S8_Miller_et_al    Cell                         Fresh   

In [35]:
choosen_global_cofounders[dataset] = ["integration_donor"]

## Braun

In [36]:
dataset = "Braun"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
adata.obs.groupby(['integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size()

integration_donor     integration_biological_unit  integration_sample_status  integration_library_platform_coarse
BRC2006_Braun_et_al   Cell                         Fresh                      3GEX                                    47021
BRC2021_Braun_et_al   Cell                         Fresh                      3GEX                                    13718
BRC2057_Braun_et_al   Cell                         Fresh                      3GEX                                    31456
BRC2061_Braun_et_al   Cell                         Fresh                      3GEX                                   128294
BRC2073_Braun_et_al   Cell                         Fresh                      3GEX                                    24790
BRC2106A_Braun_et_al  Cell                         Fresh                      3GEX                                    61996
BRC2110_Braun_et_al   Cell                         Fresh                      3GEX                                    60345
BRC2114_Braun_et_a

In [37]:
choosen_global_cofounders[dataset] = ["integration_donor"]

## Suo

In [38]:
dataset = "Suo"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
adata.obs.groupby(['integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size()

integration_donor  integration_biological_unit  integration_sample_status  integration_library_platform_coarse
F19_Suo_et_al      Cell                         Fresh                      3GEX                                    5272
                                                                           5GEX                                       0
F21_Suo_et_al      Cell                         Fresh                      3GEX                                   32833
                                                                           5GEX                                       0
F22_Suo_et_al      Cell                         Fresh                      3GEX                                   15123
                                                                           5GEX                                       0
F23_Suo_et_al      Cell                         Fresh                      3GEX                                   22415
                                                 

In [39]:
choosen_global_cofounders[dataset] = ["integration_donor", "integration_library_platform_coarse"]

## He

In [40]:
dataset = "He"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
adata.obs.groupby(['integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size()

integration_donor   integration_biological_unit  integration_sample_status  integration_library_platform_coarse
BRC2188_He_et_al    Cell                         Fresh                      5GEX                                    4203
BRC2192_He_et_al    Cell                         Fresh                      5GEX                                    4784
HDBR14804_He_et_al  Cell                         Fresh                      5GEX                                    2271
HDBR14808_He_et_al  Cell                         Fresh                      5GEX                                    2139
HDBR14853_He_et_al  Cell                         Fresh                      5GEX                                    4570
HDBR14854_He_et_al  Cell                         Fresh                      5GEX                                    2666
HDBR15167_He_et_al  Cell                         Fresh                      5GEX                                    4802
HDBR15168_He_et_al  Cell                 

In [41]:
choosen_global_cofounders[dataset] = ["integration_donor"]

## Garcia

In [42]:
dataset = "Garcia"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
with pd.option_context('display.max_rows', None):
    display(adata.obs.groupby(['integration_donor', 'integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size())

integration_donor   integration_biological_unit  integration_sample_status  integration_library_platform_coarse
F100_Garcia_et_al   Cell                         Fresh                      5GEX                                    1579
                                                 Frozen                     5GEX                                       0
F122_Garcia_et_al   Cell                         Fresh                      5GEX                                   15166
                                                 Frozen                     5GEX                                       0
F123_Garcia_et_al   Cell                         Fresh                      5GEX                                     311
                                                 Frozen                     5GEX                                       0
F126_Garcia_et_al   Cell                         Fresh                      5GEX                                     980
                                         

In [43]:
choosen_global_cofounders[dataset] = ["integration_donor", "integration_sample_status"]

## Kanemaru

In [44]:
dataset = "Kanemaru"
adata_file_path = os.path.join(tardis.config.io_directories["processed"], f"dataset_complete_{dataset}.h5ad")
assert os.path.isfile(adata_file_path), f"File not already exist: `{adata_file_path}`"
adata = ad.read_h5ad(adata_file_path, backed='r')
with pd.option_context('display.max_rows', None):
    display(adata.obs.groupby(['integration_biological_unit', 'integration_sample_status', 'integration_library_platform_coarse']).size())

integration_biological_unit  integration_sample_status  integration_library_platform_coarse
Cell                         Fresh                      3GEX                                       0
                                                        5GEX                                   45333
                             Frozen                     3GEX                                       0
                                                        5GEX                                       0
Nuclei                       Fresh                      3GEX                                       0
                                                        5GEX                                       0
                             Frozen                     3GEX                                   86303
                                                        5GEX                                       0
dtype: int64

In [45]:
with pd.option_context('display.max_rows', None):
    display(adata.obs.groupby(['integration_donor', 'integration_biological_unit']).size())

integration_donor     integration_biological_unit
C82_Kanemaru_et_al    Cell                               0
                      Nuclei                           583
C85_Kanemaru_et_al    Cell                               0
                      Nuclei                          2676
C86_Kanemaru_et_al    Cell                            5728
                      Nuclei                             0
C87_Kanemaru_et_al    Cell                               0
                      Nuclei                          4349
C92_Kanemaru_et_al    Cell                               0
                      Nuclei                         14400
C94_Kanemaru_et_al    Cell                           12415
                      Nuclei                             0
C97_Kanemaru_et_al    Cell                            9637
                      Nuclei                             0
C98_Kanemaru_et_al    Cell                               0
                      Nuclei                          2919
C99_Ka

In [46]:
choosen_global_cofounders[dataset] = ["integration_donor", "integration_biological_unit"]

# Decisisons

In [47]:
choosen_global_cofounders

{'Miller': ['integration_donor'],
 'Braun': ['integration_donor'],
 'Suo': ['integration_donor', 'integration_library_platform_coarse'],
 'He': ['integration_donor'],
 'Garcia': ['integration_donor', 'integration_sample_status'],
 'Kanemaru': ['integration_donor', 'integration_biological_unit']}